In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss



In [45]:
data = pd.read_json("/home/nlp/Downloads/train.json","r")

In [46]:
data.head(2)

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue


In [47]:
data["num_photos"] = data["photos"].apply(len)
data["num_features"] = data["features"].apply(len)
data["num_decription"] = data["description"].apply(lambda x: len(x.split(" ")))
data["created"] = pd.to_datetime(data['created'])
data["created_day"] = data["created"].dt.day
data["created_month"] = data["created"].dt.month
data["created_year"] = data["created"].dt.year

In [48]:
num_features = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_decription",
             "created_year", "created_month", "created_day"]
X = data[num_features]
y = data["interest_level"]
X.head()



,bathrooms,bedrooms,latitude,longitude,price,num_photos,num_features,num_decription,created_year,created_month,created_day
10,1.5,3,40.7145,-73.9425,3000,5,0,95,2016,6,24
10000,1.0,2,40.7947,-73.9667,5465,11,5,9,2016,6,12
100004,1.0,1,40.7388,-74.0018,2850,8,4,94,2016,4,17
100007,1.0,1,40.7539,-73.9677,3275,3,2,80,2016,4,18
100013,1.0,4,40.8241,-73.9493,3350,3,1,68,2016,4,28


In [49]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [50]:
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

0.62740574132043891

In [51]:
test_data = pd.read_json("/home/nlp/Downloads/test.json",'r')
test_data["num_photos"] = test_data["photos"].apply(len)
test_data["num_features"] = test_data["features"].apply(len)
test_data["num_decription"] = test_data["description"].apply(lambda x: len(x.split(" ")))
test_data["created"] = pd.to_datetime(test_data['created'])
test_data["created_day"] = test_data["created"].dt.day
test_data["created_month"] = test_data["created"].dt.month
test_data["created_year"] = test_data["created"].dt.year


In [52]:
X = data[num_features]

y = clf.predict_proba(X)

In [53]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}
labels2idx



{u'high': 0, u'low': 1, u'medium': 2}

In [54]:
sub = pd.DataFrame()
sub["listing_id"] = data["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("sub2.csv", index=False)

